In [3]:
import requests
import re
import json
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

city = 'minneapolis/' #*****change this city to what you want!!!!*****

#just grabbing the first 20 pages
#feel free to make this prettier
url1 = 'https://www.zillow.com/homes/for_sale/'+city
url2 = 'https://www.zillow.com/homes/for_sale/'+city+'/2_p/'
url3 = 'https://www.zillow.com/homes/for_sale/'+city+'/3_p/'
url4 = 'https://www.zillow.com/homes/for_sale/'+city+'/4_p/'
url5 = 'https://www.zillow.com/homes/for_sale/'+city+'/5_p/'
url6 = 'https://www.zillow.com/homes/for_sale/'+city+'/6_p/'
url7 = 'https://www.zillow.com/homes/for_sale/'+city+'/7_p/'
url8 = 'https://www.zillow.com/homes/for_sale/'+city+'/8_p/'
url9 = 'https://www.zillow.com/homes/for_sale/'+city+'/9_p/'
url10 = 'https://www.zillow.com/homes/for_sale/'+city+'/10_p/'

#add headers in case you use chromedriver (captchas are no fun); namely used for chromedriver
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

with requests.Session() as s:
    r1 = s.get(url1, headers=req_headers)
    r2 = s.get(url2, headers=req_headers)
    r3 = s.get(url3, headers=req_headers)
    r4 = s.get(url4, headers=req_headers)
    r5 = s.get(url5, headers=req_headers)
    r6 = s.get(url6, headers=req_headers)
    r7 = s.get(url7, headers=req_headers)
    r8 = s.get(url8, headers=req_headers)
    r9 = s.get(url9, headers=req_headers)
    r10 = s.get(url10, headers=req_headers)

    data1 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r1.text).group(1))
    data2 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r2.text).group(1))
    data3 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r3.text).group(1))
    data4 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r4.text).group(1))
    data5 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r5.text).group(1))
    data6 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r6.text).group(1))
    data7 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r7.text).group(1))
    data8 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r8.text).group(1))
    data9 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r9.text).group(1))
    data10 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r10.text).group(1))

data_list = [data1,data2,data3,data4,data5,data6,data7,data8,data9,data10]
            # ,data11,data12,data13,data14,data15,data16,data17,data18,data19,data20]

df = pd.DataFrame()

def make_frame(frame):
    for i in data_list:
        for item in i['cat1']['searchResults']['listResults']:
            frame = frame.append(item, ignore_index=True)
    return frame

df = make_frame(df)
    
#drop cols
df = df.drop('hdpData', 1) #remove this line to see a whole bunch of other random cols, in dict format

#drop dupes
df = df.drop_duplicates(subset='zpid', keep="last")

#filters
df['zestimate'] = df['zestimate'].fillna(0)
df['best_deal'] = df['unformattedPrice'] - df['zestimate']
df = df.sort_values(by='best_deal',ascending=True)

print('shape:', df.shape)
display(df[['id','address','beds','baths','area','price','zestimate','best_deal']].head(20))

shape: (400, 50)


,id,address,beds,baths,area,price,zestimate,best_deal
224,88836811,"215 10th Ave S UNIT 617, Minneapolis, MN 55415",2,2.0,1395,"$400,000",468200,-68200
300,1719338,"4119 Colfax Ave N, Minneapolis, MN 55412",3,1.0,1356,"$160,000",226000,-66000
390,2009692,"2316 Harriet Ave, Minneapolis, MN 55405",4,2.0,2372,"$339,900",395807,-55907
22,1849489,"5320 Penn Ave S, Minneapolis, MN 55419",4,2.0,2748,"$599,900",646500,-46600
157,82651159,"2500 Blaisdell Ave APT 401, Minneapolis, MN 55404",1,1.0,686,"$59,900",102100,-42200
49,1977406,"3901 E 26th St, Minneapolis, MN 55406",3,1.0,1329,"$325,000",363300,-38300
162,1846739,"5329 47th Ave S, Minneapolis, MN 55417",3,2.0,1480,"$359,900",392600,-32700
344,1696928,"3335 Tyler St NE, Minneapolis, MN 55418",4,2.0,1700,"$350,000",381800,-31800
156,1812683,"643 Pierce St NE, Minneapolis, MN 55413",3,2.0,1240,"$325,000",352900,-27900
36,1820041,"5306 Elliot Ave, Minneapolis, MN 55417",3,1.0,1328,"$325,000",351700,-26700
